In [2]:
import pandas as pd
df = pd.read_excel('test_case1.xlsx')

In [3]:
df.head()

,product_id,quantity,purchase_date,current_date,subscription_date,user_id,Cohort
0,1week_9.99_USD_subscription,1,2021-02-19,2021-02-21,2021-02-19,9484222,1
1,1week_9.99_USD_subscription,1,2021-02-17,2021-02-21,2021-02-17,9422402,1
2,1week_9.99_USD_subscription,1,2021-02-18,2021-02-21,2021-02-18,4997991,1
3,1week_9.99_USD_subscription,1,2021-02-18,2021-02-21,2021-02-18,9455582,1
4,1week_9.99_USD_subscription,1,2021-02-16,2021-02-21,2021-01-26,8803657,4


In [4]:
def explore(x):
    divider = "*_*"
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    
    print("Dataframe Makeup \n") #title for output.
    
    x.info() # Explains what the data and values the data is madeup from.
    
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    
    print("Descriptive Statistics \n\n", x.describe().round(2)) #Gives a statstical breakdown of the data.
    
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    
    print("Shape of dataframe: {}".format(x.shape)) # Gives the shape of the data.
    
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    return
explore(df)


 *_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_* 

Dataframe Makeup 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51146 entries, 0 to 51145
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   product_id         51146 non-null  object        
 1   quantity           51146 non-null  int64         
 2   purchase_date      51146 non-null  datetime64[ns]
 3   current_date       51146 non-null  datetime64[ns]
 4   subscription_date  51146 non-null  datetime64[ns]
 5   user_id            51146 non-null  int64         
 6   Cohort             51146 non-null  int64         
dtypes: datetime64[ns](3), int64(3), object(1)
memory usage: 2.7+ MB

 *_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_* 

Descriptive Statistics 

        quantity                  purchase_date         current_date  \
count   51146.0                          51146                51146   
mean       

Перевірка вмісту 

In [5]:
print('unique users ',df['user_id'].nunique())
print('unique quantities ',df['quantity'].nunique())
print('unique cohorts ',df['Cohort'].nunique())

unique users  19536
unique quantities  1
unique cohorts  7


In [6]:
# Додавання колонки з кількістю тижнів від дати першої підписки до дати операції
df['Weeks_since_subscription'] = ((df['purchase_date'] - df['subscription_date']).dt.days // 7) + 1
df.head()

,product_id,quantity,purchase_date,current_date,subscription_date,user_id,Cohort,Weeks_since_subscription
0,1week_9.99_USD_subscription,1,2021-02-19,2021-02-21,2021-02-19,9484222,1,1
1,1week_9.99_USD_subscription,1,2021-02-17,2021-02-21,2021-02-17,9422402,1,1
2,1week_9.99_USD_subscription,1,2021-02-18,2021-02-21,2021-02-18,4997991,1,1
3,1week_9.99_USD_subscription,1,2021-02-18,2021-02-21,2021-02-18,9455582,1,1
4,1week_9.99_USD_subscription,1,2021-02-16,2021-02-21,2021-01-26,8803657,4,4


In [7]:
# Побудова зведеної таблиці для когортного аналізу
cohort_pivot = df.pivot_table(
    index='Cohort',  # Когорта - номер тижня підписки
    columns='Weeks_since_subscription',  # Тижні з моменту підписки
    values='user_id',  # Кількість клієнтів з покупками
    aggfunc='count'
)

cohort_pivot

Weeks_since_subscription,1,2,3,4,5,6,7
Cohort,,,,,,,
1,2820.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3184.0,1931.0,NaN,NaN,NaN,NaN,NaN
3,2897.0,2019.0,1401.0,NaN,NaN,NaN,NaN
4,3125.0,2286.0,1893.0,1371.0,NaN,NaN,NaN
5,3341.0,2564.0,2145.0,1865.0,1368.0,NaN,NaN
6,3390.0,2616.0,2250.0,1953.0,1778.0,1354.0,NaN
7,792.0,642.0,560.0,507.0,458.0,422.0,214.0


Беремо кількість нових клієнтів за 100% та кумулятивно рахуємо кількість покупок за періоди 

In [8]:
cohort_pivot_cumulative = cohort_pivot.cumsum(axis=1)
# Делаем деление каждого элемента на первый элемент строки
cohort_pivot_normalized = cohort_pivot_cumulative.div(cohort_pivot_cumulative.iloc[:, 0], axis=0)
cohort_pivot_normalized

Weeks_since_subscription,1,2,3,4,5,6,7
Cohort,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.606470,NaN,NaN,NaN,NaN,NaN
3,1.0,1.696928,2.180532,NaN,NaN,NaN,NaN
4,1.0,1.731520,2.337280,2.776000,NaN,NaN,NaN
5,1.0,1.767435,2.409458,2.967674,3.377133,NaN,NaN
6,1.0,1.771681,2.435398,3.011504,3.535988,3.935398,NaN
7,1.0,1.810606,2.517677,3.157828,3.736111,4.268939,4.539141


Знаходимо середню кількість покупок в кожен період

In [9]:
average_values = cohort_pivot_normalized.mean()
average_values

Weeks_since_subscription
1    1.000000
2    1.730773
3    2.376069
4    2.978252
5    3.549744
6    4.102169
7    4.539141
dtype: float64

Та LTV, множачи це на чек

In [10]:
ltv = average_values * 9.99
ltv

Weeks_since_subscription
1     9.990000
2    17.290426
3    23.736929
4    29.752735
5    35.461942
6    40.980666
7    45.346023
dtype: float64

In [11]:
cohort_pivot_normalized.loc['Average'] = average_values
cohort_pivot_normalized.loc['LTV'] = ltv
cohort_pivot_normalized

Weeks_since_subscription,1,2,3,4,5,6,7
Cohort,,,,,,,
1,1.00,NaN,NaN,NaN,NaN,NaN,NaN
2,1.00,1.606470,NaN,NaN,NaN,NaN,NaN
3,1.00,1.696928,2.180532,NaN,NaN,NaN,NaN
4,1.00,1.731520,2.337280,2.776000,NaN,NaN,NaN
5,1.00,1.767435,2.409458,2.967674,3.377133,NaN,NaN
6,1.00,1.771681,2.435398,3.011504,3.535988,3.935398,NaN
7,1.00,1.810606,2.517677,3.157828,3.736111,4.268939,4.539141
Average,1.00,1.730773,2.376069,2.978252,3.549744,4.102169,4.539141
LTV,9.99,17.290426,23.736929,29.752735,35.461942,40.980666,45.346023


In [12]:
print(f"LTV for 1.5 Months (6 Weeks) = 40.980666")

LTV for 1.5 Months (6 Weeks) = 40.980666


Для виконання наступного завдання було спрогнозовано середню кількість покупок в кожному тижні

In [13]:
import numpy as np

# Початкові значення покупок в кожному тижні
start_values = np.array([1.000000, 1.730773, 2.376069, 2.978252, 3.549744, 4.102169, 4.539141])

# прогнозовані значення
forecast_values = np.zeros(25)

# Початкові значення для прогнозу
forecast_values[0:len(start_values)] = start_values

# Прогнозуємо на основі линійної екстраполяції
for i in range(len(start_values), 25):
    forecast_values[i] = forecast_values[i-1] + (forecast_values[i-1] - forecast_values[i-2])

# Результати - в таблицю
forecast_table = pd.DataFrame({
    'Week': np.arange(1, 26),
    'Forecast': forecast_values
})

forecast_table


,Week,Forecast
0,1,1.000000
1,2,1.730773
2,3,2.376069
3,4,2.978252
4,5,3.549744
5,6,4.102169
6,7,4.539141
7,8,4.976113
8,9,5.413085
9,10,5.850057


І таким же чином як і для 6 тижнів знайдено LTV

In [14]:
print(f"LTV for 2 Months (8 Weeks) = {forecast_values[7] * 9.99}")
print(f"LTV for 5 Months (20 Weeks) = {forecast_values[19] * 9.99}")
print(f"LTV for 6 Months (24 Weeks) = {forecast_values[23] * 9.99}")

LTV for 2 Months (8 Weeks) = 49.71136887
LTV for 5 Months (20 Weeks) = 102.09557223000004
LTV for 6 Months (24 Weeks) = 119.55697335000008


Так як  ROAS = (total customers * LTV)/campaign value, а в нашому випадку ROAS = (total customers * LTV)/(total customers * customer acquisition value) = LTV/customer acquisition

In [15]:
print(f"ROAS for 6 Weeks = {(forecast_values[5] * 9.99)/30}")
print(f"ROAS for 6 Months = {(forecast_values[23] * 9.99)/30}")

ROAS for 6 Weeks = 1.366022277
ROAS for 6 Months = 3.9852324450000025


Варто відзначити, що продукт стає прибутковим лише на 5 ому тижні, що не є дуже гарною тенденцією

In [18]:
print(f"ROAS for 4 Weeks = {(forecast_values[3] * 9.99)/30}")

ROAS for 4 Weeks = 0.991757916
